In [1]:
pip install opencv-python

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np

### Capture Video Stream

In [3]:
cap = cv2.VideoCapture('Video/face_track.mp4')

### Tale first frame of the video

In [4]:
ret, frame = cap.read()

### Set up the initial tracking window

In [5]:
face_casc=cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
face_rects=face_casc.detectMultiScale(frame)

###here i'm Converting the list to a Tuple

In [6]:
face_x,face_y,w,h = tuple(face_rects[0])
track_window=(face_x,face_y,w,h)

### here I m doing a Set up the ROI for Tracking

In [7]:
roi = frame[face_y:face_y+h,
           face_x:face_x+w]

### the HSV color maping

In [8]:
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

### here is the Histogram to target on each frame for the meanshift calculation and i'll guide you to do this

In [9]:
roi_hist=cv2.calcHist([hsv_roi],
                     [0],
                     None,
                     [180],
                     [0,180])

### Normalize the histogram


In [10]:
cv2.normalize(roi_hist,
             roi_hist,
             0,
             255,
             cv2.NORM_MINMAX);


### Set the termination criteria

10 iterations or move 1 pt

In [11]:
term_crit=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

In [12]:

while(True):
    ret, frame=cap.read()
    if ret == True:
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        #so here i'm calculating the base of ROI
        dest_roi = cv2.calcBackProject([hsv],
                                      [0],
                                      roi_hist,
                                      [0,100],
                                      1)
        #here i'm using the meanshift function to get the new coordinates of the rectangle ( you can do it manually by creating a self function to calculate this rectangle )
        ret, track_window = cv2.meanShift(dest_roi,
                                         track_window,
                                         term_crit)
        #here i'm drawing a new rectangle on image and then i opened new window and display
        x,y,w,h = track_window   
        img2= cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),3)
        cv2.imshow('FireTracker',img2)
        if cv2.waitKey(50)& 0xFF == 27:
            break  
    else:
        break
cap.release()
cv2.destroyAllWindows()